# Задача извлечения нецензурной лексики из текста

### Импорт библиотек

In [2]:
import pandas as pd
import numpy as np

from pathlib import Path
import os

from concurrent.futures import ProcessPoolExecutor, as_completed

from typing import List, Set

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import torch

seed = 42
np.random.seed(seed)

### Импорт моделей

In [ ]:
# Data preprocessing
from models.data_preprocessor import SlurDataPreprocessor, SlurDataset
# Baselines
from models.dictionary_extractor import DictionaryExtractor
from models.regex_extractor import RegexExtractor
# BERT-based models
from models.transformer_extractor import (
    RuBertSlurExtractor,
    RoSBertaSlurExtractor,
)

# Извлечение и анализ данных

## Загрузка данных

In [ ]:
data_dir = Path("data")
output_dir = Path("./")

train_df = pd.read_csv(data_dir / "train.csv")
test_df = pd.read_csv(data_dir / "test.csv")

### Словарь матов

In [ ]:
with open(data_dir / "ru_curse_words.txt", 'r', encoding='utf-8') as f:
    curse_words = set(word.strip().lower() for word in f.readlines() 
                      if word.strip())

In [ ]:
# Расширение словаря матами из train данных
def extract_words_from_labels(df, label_col='labels') -> Set[str]:
    """Извлекает все уникальные слова из столбца labels"""
    slurs = set()
    
    slur_strs = df[label_col].dropna()
    slurs = set(slur for slurs in slur_strs for slur in slurs.strip().split(',') if slur)
    
    return slurs

Дополнение словаря матов из train данных

In [ ]:
train_curse_words = extract_words_from_labels(train_df, 'label')
print(f"Примеры из train: {list(train_curse_words)[:10]}")

# Объединяем словари
ext_curse_words = curse_words.union(train_curse_words)
print(f"\nРасширенный словарь:")
print(f"Исходный словарь: {len(curse_words)} слов")
print(f"Найдено матов в train данных: {len(train_curse_words)} слов")
print(f"Итоговый словарь: {len(ext_curse_words)} слов")
print(f"Добавлено новых слов: {len(ext_curse_words) - len(curse_words)}")


Примеры из train: ['пизджак', 'задолбала', 'хуйня ебаная;', 'хуёвая', 'ахуенн*я', 'ебланы', 'уебали', 'на*бщиков', 'с*** б****', 'ахун']

Расширенный словарь:
Исходный словарь: 2297 слов
Найдено матов в train данных: 3687 слов
Итоговый словарь: 5713 слов
Добавлено новых слов: 3416


## Анализ и разбиение данных

In [ ]:
print(f"Train: {train_df.shape}")
print(f"Test: {test_df.shape}")

Train: (244739, 3)
Test: (66949, 2)


### Анализ матов

In [ ]:
print("Анализ поля 'label':")

train_df['has_slur'] = train_df['label'].notna() & (train_df['label'].str.strip() != '')
print(f"Доля строк с матами: {train_df['has_slur'].mean():.1%}")

# Статистика по количеству матов в отзыве
slur_lengths = train_df['label'].map(lambda x: len(str(x).split(',')) if pd.notna(x) and str(x) != '' else 0)
print(f"\nСтатистика количества матов в отзыве:")
print(slur_lengths.describe())

Анализ поля 'label':
Всего записей: 244739
Пустых значений: 209888

Статистика количества матов в ответе:
count    244739.000000
mean          0.162965
std           0.442486
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max          16.000000
Name: label, dtype: float64


### Разбиение на X и Y

In [ ]:
target_column = "label"

val_size = 0.1  # 0.1 так как есть ещё тестовый датасет
X_train, X_val, y_train, y_val = train_test_split(
    train_df['text'],
    train_df[target_column],
    test_size=val_size,
    random_state=seed
)

print(f"Train : {X_train.shape} {y_train.shape}")
print(f"Validation : {X_val.shape} {y_val.shape}")
print(f"\nРаспределение классов в train: {y_train.value_counts()}")
print(f"Распределение классов в val: {y_val.value_counts()}")

Train : (220265,) (220265,)
Validation : (24474,) (24474,)

Распределение классов в train: label
говно                 4623
херня                 2094
гавно                 2062
дерьмо                1694
хуйня                  882
                      ... 
коноёбило                1
уёбищная,говно           1
бляя,бляб,блят,бля       1
заебали,нехуя,хуя        1
хуйовое                  1
Name: count, Length: 4521, dtype: int64
Распределение классов в val: label
говно                        523
гавно                        230
херня                        223
дерьмо                       190
хуйня                         95
                            ... 
въебали                        1
заклейте себе очко,ебучий      1
хуйнч                          1
ахуеннн                        1
задолбались                    1
Name: count, Length: 934, dtype: int64


### Просмотр данных

In [ ]:
pd.set_option('display.max_colwidth', None)  # Показать полную ширину колонок
pd.set_option('display.width', None)  # Убрать ограничение по ширине

# Извлечём первые 10 строк с непустым label
train_df[train_df['label'].notna()].set_index('ID').head(10)

# Бейслайны

## Словарный поиск

In [ ]:
SRCH_METHODS = DictionaryExtractor.SRCH_METHODS


def eval_levenst(dist):
    model = DictionaryExtractor(ext_curse_words, method=SRCH_METHODS.levenshtein, allowed_dist=dist)
    model.fit(X_train[:5000], y_train[:5000])
    model.evaluate(X_val[:1000], y_val[:1000], name=f"Levenshtein (dist={dist})")


def test_dict_extractor_params():
    """Проверка работы с различными параметрами"""
    for method in SRCH_METHODS._fields:
        extractor = DictionaryExtractor(ext_curse_words, method=method)

        if method == SRCH_METHODS.levenshtein:
            # Для levenshtein нужно задать допустимое расстояние
            distances = list(np.arange(0.5, 5, 0.5))
            cpu_count = max(os.cpu_count() - 2, 1)
            print("Using {} cores".format(cpu_count))
            with ProcessPoolExecutor(10) as executor:
                future_to_dist = {executor.submit(eval_levenst, dist): dist for dist in distances}
                
        else:
            extractor.fit(X_train, y_train)
            print(f"Testing {method}\n")
            extractor.evaluate(X_val[:1000], y_val[:1000])

        print('-' * 40)

In [ ]:
# test_dict_extractor_params()

Фактически метод **точного совпадения** показывает лучшие результаты (3.03). Его возьмём в качестве бейслайна

In [ ]:
dict_model = DictionaryExtractor(ext_curse_words, method=SRCH_METHODS.match)
_ = dict_model.fit(X_train, y_train)

In [ ]:
_ = dict_model.evaluate(X_val, y_val, name=f"Exact match DictionaryExtractor")

Evaluating Exact match DictionaryExtractor: 100%|██████████| 24474/24474 [00:00<00:00, 1364990.71it/s]

Exact match DictionaryExtractor - Average Levenshtein Distance: 3.0819


## Поиск с регулярными выражениями

In [ ]:
regex_extractor = RegexExtractor(ext_curse_words)
# _ = regex_extractor.fit(X_train, y_train)R

In [ ]:
# predictions = regex_extractor.transform(X_val)

In [ ]:
'''
20 минут. Avg Levenshtein: 671.8156
'''
# _ = regex_extractor.evaluate(predicts=predictions, true_slurs=y_val, name="RegexExtractor")

'\n20 минут. Avg Levenshtein: 671.8156\n'

# Пайплайн

In [ ]:
class SlurPipeline:
    def __init__(self, model_class, output_dir, **model_kwargs):
        self.model_class = model_class
        self.output_dir = output_dir
        self.model_kwargs = model_kwargs

        train_args = {
            'batch_size': model_kwargs.pop('batch_size', 32),
            'epochs': model_kwargs.pop('epochs', 3),
            'learning_rate': model_kwargs.pop('learning_rate', 1e-5),
            'warmup_steps': model_kwargs.pop('warmup_steps', 200),
            'weight_decay': model_kwargs.pop('weight_decay', 0.01),
            'logging_steps': model_kwargs.pop('logging_steps', 50),
            'grad_accum_steps': model_kwargs.pop('grad_accum_steps', 1),
            'max_length': model_kwargs.pop('max_length', 128),
        }

        # Создаем модель для получения токенизатора
        self.model = model_class(output_dir=output_dir, train_args=train_args, **model_kwargs)
        self.preprocessor = SlurDataPreprocessor(max_length=train_args['max_length'], verbose=True)

    def fit(self, X, y):
        """Обучение"""
        # Подготавливаем данные
        if hasattr(X, 'values'):
            texts = X.values.tolist()
        else:
            texts = list(X)

        if hasattr(y, 'values'):
            labels = y.values.tolist()
        else:
            labels = list(y)

        # Создаем training data
        training_data = self.preprocessor.prepare_training_data(
            texts, labels, self.model.tokenizer
        )

        # разбиваем на два списка
        texts_l = [d['text']  for d in training_data]
        spans_l = [d['spans'] for d in training_data]

        # Обучаем
        self.model.fit(texts_l, spans_l)

        return self

    def predict(self, X):
        """Предсказание"""
        if hasattr(X, 'values'):
            texts = X.values.tolist()
        else:
            texts = list(X)

        return self.model.transform(texts)

    def transform(self, X):
        return self.predict(X)

    def evaluate(self, y_pred, y_true, name=None):
        model_name = name if name else self.model.__class__.__name__
        return self.model.evaluate(predicts=y_pred, true_slurs=y_true, name=model_name)

# Модели

## RuBERT

In [ ]:
# rubert_pipeline = SlurPipeline(
#     RuBertSlurExtractor,
#     output_dir="rubert_extractor",
#     batch_size=64,
# )

In [ ]:
# _ = rubert_pipeline.fit(X_train[:50000], y_train[:50000])

In [ ]:
# val_samples_k = 10000
# predictions = rubert_pipeline.transform(X_val[:val_samples_k])

## RoSBERTa

In [ ]:
rosberta_pipeline = SlurPipeline(
    RoSBertaSlurExtractor,
    output_dir=output_dir / "rosberta_extractor",
    batch_size=32,
    epochs=2,
    learning_rate=2e-5,
    grad_accum_steps=2,
    warmup_steps=200
)

In [ ]:
rosberta_pipeline.fit(X_train, y_train)

In [ ]:
val_samples_k = 50
predictions = rosberta_pipeline.transform(X_val[:val_samples_k])

Посмотрим на примеры предсказаний

In [ ]:
comp = pd.DataFrame({'Text': X_val[:val_samples_k], 'Prediction': predictions, 'True': y_val[:val_samples_k].values})
comp[comp['Prediction'] != ''][:10]

In [ ]:
rosberta_pipeline.evaluate(y_pred=predictions, y_true=y_val[:val_samples_k], name='RoSBERTa')

## IlyaGusev's RuBERT toxic editor

In [ ]:
# gusev_rb_pipeline = SlurPipeline(
#     RuBertLoRASlurExtractor,
#     output_dir=output_dir / "gusev_rubert_extractor",
#     batch_size=32,
#     epochs=2,
#     learning_rate=2e-5,
#     grad_accum_steps=2,
#     warmup_steps=200
# )

In [ ]:
# _ = gusev_rb_pipeline.fit(X_train[:50], y_train[:50])

In [ ]:
# val_samples_k = 5000
# predictions = gusev_rb_pipeline.transform(X_val[:val_samples_k])

In [ ]:
# comp = pd.DataFrame({'Text': X_val[:val_samples_k], 'Prediction': predictions, 'True': y_val[:val_samples_k].values})
# comp[comp['Prediction'] != ''][:10]

# Выгрузка решения

In [ ]:
def create_submission(model, test_data, filename="submission.csv"):
    """
    Создает файл submission для отправки результатов

    Args:
        model: обученная модель с методом predict
        test_data: DataFrame с тестовыми данными
        filename: имя файла для сохранения
    """

    # Получаем предсказания на тестовых данных
    test_predictions = model.predict(test_data['text'])

    # Создаем DataFrame для submission
    submission = pd.DataFrame({
        'ID': test_data['ID'],  # Используем ID из тестовых данных
        'label': test_predictions
    })

    # Сохраняем файл
    submission.to_csv(filename, index=False)

    print(f"Файл {filename} создан!")
    print(f"Размер: {submission.shape}")
    print(f"Распределение предсказаний:")
    print(submission['label'].value_counts().sort_index())
    print(f"\nПример submission:")
    print(submission.head(10))

    return submission


In [ ]:
# submission = create_submission(
#     model=rosberta_pipeline,
#     test_data=test_df[:50],
#     filename=output_dir / "RoSBERTa_v2.7_submission.csv"
# )

## Сохранение/загрузка модели

In [ ]:
import torch
from pathlib import Path


def save_model(pipeline, model_name, save_dir="saved_models"):
    save_path = Path(save_dir) / model_name
    save_path.mkdir(parents=True, exist_ok=True)

    print(f"Сохраняем модель {model_name} в {save_path}")
    trainer = pipeline.model.trainer

    torch.save({
        'model_state_dict': trainer.model.state_dict(),
    }, save_path / "trainer_state.pt")

    print("Модель сохранена:")

    return save_path

def load_checkpoint(save_path, filename="trainer_state.pt"):
    save_path = Path(save_path)

    print("Загружаем чекпоинт")

    ckpt = torch.load(save_path / filename, weights_only=False)

    print("Загружено")

    return ckpt

In [ ]:
checkpoint_dir = "./rosberta_v2.7" # '/kaggle/input/rosberta-slurs/pytorch/v2.3/3/'
loaded_checkpoint = load_checkpoint(checkpoint_dir, 'trainer_state.pt')
# save_path = save_model(rosberta_pipeline, "rosberta_v2.6", output_dir)

# Дообучение своих моделей

Проверяем, что загруженная модель работает корректно

In [ ]:
# Новая модель, куда будут загружены старые настройки
rosberta27_pipeline = SlurPipeline(
    FineTuneRoSBERTa,  # Класс для предобученной модели
    output_dir=output_dir / "rosberta27_pipeline",
    model_saved_dict=loaded_checkpoint,
    batch_size=64,
    epochs=1,
    learning_rate=3e-5,
    grad_accum_steps=1,
    warmup_steps=200,
    max_length=200  # макс количество токенов
)
rosberta27_pipeline.model.is_fitted = True

Устройство: cuda
Загружаем модель: ai-forever/ru-en-RoSBERTa


Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at ai-forever/ru-en-RoSBERTa and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Модель загружена


In [ ]:
val_samples_k = 500
predictions = rosberta27_pipeline.transform(X_val[:val_samples_k])

Extracting slurs: 100%|██████████| 500/500 [00:19<00:00, 25.99it/s]


In [ ]:
comp = pd.DataFrame({'Text': X_val[:val_samples_k], 'Prediction': predictions, 'True': y_val[:val_samples_k].values})
comp[comp['Prediction'].str.contains(',')][:50]

### Обучение

In [ ]:
_ = rosberta27_pipeline.fit(pd.concat([X_train[50000:], X_val]),
                            pd.concat([y_train[50000:], y_val]))

In [ ]:
val_samples_k = 5000
predictions = rosberta27_pipeline.transform(X_val[:val_samples_k])

In [ ]:
rosberta27_pipeline.evaluate(y_pred=predictions, y_true=y_val[:val_samples_k], name='RoSBERTa_v2.6')

Evaluating RoSBERTa_v2.6: 100%|██████████| 5000/5000 [00:00<00:00, 610382.44it/s]

RoSBERTa_v2.6 - Average Levenshtein Distance: 2.8862


2.8862

In [ ]:
comp = pd.DataFrame({'Text': X_val[:val_samples_k], 'Prediction': predictions, 'True': y_val[:val_samples_k].values})
comp[comp['Prediction'] != ''][:10]

In [ ]:
submission = create_submission(
    model=rosberta27_pipeline,
    test_data=test_df,
    filename=output_dir / "RoSBERTa_v2.7.1_submission.csv"
)

Extracting slurs:  57%|█████▋    | 38262/66949 [21:42<20:00, 23.89it/s]

In [ ]:
save_path = save_model(rosberta27_pipeline, "rosberta_v2.7", output_dir)